 # Cours: Applied Data Science Capstone
## Section 0 : 
### The section 0 of this notebook is created for Capstone_Coursera project as just 
### the first test of notebooke creation and check sharing in GiyHub
### Other sections will be following the cours amendes
 

In [12]:
import pandas as pd
import numpy as np

In [13]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


 # Cours: Applied Data Science Capstone
## section 1: 
### Segmenting and Clustering Neighborhoods in Toronto


### section 1-1
#### Transfer data of Toronto's  neighborhoods  from wikipedia to a dataframe

In [14]:
import pandas as pd
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
df_TNeib = pd.read_html(url, attrs={"class": "wikitable"})[0] # 0 is for the 1st table in this particular page
df_TNeib.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


### Section 1-2
#### Ignore cells with a borough that is Not assigned

In [15]:
df_TNeib.drop(df_TNeib.index[df_TNeib['Borough'] == 'Not assigned'], inplace = True)
df_TNeib.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


### Section 1-3
#### More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11  in the above table.

In [16]:
print ("this section is not applicable as the latest version \n of wikipedia is in the same format as we need.")

this section is not applicable as the latest version 
 of wikipedia is in the same format as we need.


### Section 1-4:
#### 1-4-1 If a cell has a borough but a Not assigned  neighborhood, then the neighborhood will be the same as the borough.
#### 1-4-2 Reset index

In [17]:
df_TNeib['Neighbourhood'] = np.where(df_TNeib['Neighbourhood']=='Not assigned', df_TNeib['Borough'], df_TNeib['Neighbourhood'])
df_TNeib.reset_index(drop=True, inplace=True)
df_TNeib.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


### Section 1-5:
#### Shape of dataframe


In [18]:
df_TNeib.shape

(103, 3)

# Cours: Applied Data Science Capstone
## Section 2
### Section 2-1 
#### Read csv file of postalcodes

In [19]:
df_PCod = pd. read_csv('https://cocl.us/Geospatial_data')
df_PCod.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [20]:
df_PCod.shape

(103, 3)

### Section 2-2 
#### Creat a data frame which connects df_Neib to related Latitude and longitude for every neighbourhood

In [22]:
df_TNeib_LL = pd.merge(df_TNeib,
                 df_PCod[['Postal Code', 'Latitude', 'Longitude']],
                 on='Postal Code')
df_TNeib_LL.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
